# Implement LSH from scratch

In this assignment, you will implement LSH from scratch and predict the labels of the test data. You will then verify the correctness of the your implementation using a "grader" function/cell (provided by us) which will match your implmentation.

The grader fucntion would help you validate the correctness of your code. 

Please submit the final Colab notebook in the classroom ONLY after you have verified your code using the grader function/cell.


**NOTE: DO NOT change the "grader" functions or code snippets written by us.Please add your code in the suggested locations.**

Ethics Code:
1. You are welcome to read up online resources to implement the code. 
2. You can also discuss with your classmates on the implmentation over Slack.
3. But, the code you wirte and submit should be yours ONLY. Your code will be compared against other stduents' code and online code snippets to check for plagiarism. If your code is found to be plagiarised, you will be awarded zero-marks for all assignments, which have a 10% wieghtage in the final marks for this course.

## Reading the data from csv file

In [1]:
# Code to mount google drive in case you are loading the data from your google drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [4]:
# Loading data from csv file
import pandas as pd
data_path = r'/content/drive/MyDrive/AAIC Assignments/Custom LSH/lsh_assignment_data.csv'

df = pd.read_csv(data_path)
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...


In [5]:
# Data Overiview
df['category'].value_counts()

sport            509
business         508
politics         415
tech             399
entertainment    384
Name: category, dtype: int64

In [6]:
df[df['category'].isna()]

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


### Creating Train and Test Datasets

Note that the labels for test data will not be present in the dataset and hence they are mentioned as NaN.

In [7]:
# The last 10 rows in the csv file are query points, so loading them into test data.
# And loading the reamining points to train_data for which labels are given.

train_data = df.iloc[:-10,:]
test_data = df.iloc[-10:,:]

In [10]:
# For train_data here the labels are in the column named "category".
train_data

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2210,politics,teens know little of politics teenagers ques...
2211,entertainment,lopez misses uk charity premiere jennifer lope...
2212,business,christmas shoppers flock to tills shops all ov...
2213,tech,progress on new internet domains by early 2005...


In [9]:
test_data

,category,text
2215,NaN,junk e-mails on relentless rise spam traffic i...
2216,NaN,top stars join us tsunami tv show brad pitt r...
2217,NaN,rings of steel combat net attacks gambling is ...
2218,NaN,davies favours gloucester future wales hooker ...
2219,NaN,beijingers fume over parking fees choking traf...
2220,NaN,cars pull down us retail figures us retail sal...
2221,NaN,kilroy unveils immigration policy ex-chatshow ...
2222,NaN,rem announce new glasgow concert us band rem h...
2223,NaN,how political squabbles snowball it s become c...
2224,NaN,souness delight at euro progress boss graeme s...


In [11]:
#Unique category Values
train_data['category'].unique()

array(['tech', 'business', 'sport', 'entertainment', 'politics'],
      dtype=object)

# DataFrame and Label split on Train and Test Data

In [12]:
y_train = train_data['category']
x_train = train_data.drop(columns=['category'])
y_test = test_data['category']
x_test = test_data.drop(columns=['category'])

In [13]:
x_train.shape,y_train.shape

((2215, 1), (2215,))

In [14]:
x_test.shape,y_test.shape

((10, 1), (10,))

In [15]:
x_train.head()

,text
0,tv future in the hands of viewers with home th...
1,worldcom boss left books alone former worldc...
2,tigers wary of farrell gamble leicester say ...
3,yeading face newcastle in fa cup premiership s...
4,ocean s twelve raids box office ocean s twelve...


In [16]:
x_test.head()

,text
2215,junk e-mails on relentless rise spam traffic i...
2216,top stars join us tsunami tv show brad pitt r...
2217,rings of steel combat net attacks gambling is ...
2218,davies favours gloucester future wales hooker ...
2219,beijingers fume over parking fees choking traf...


## Custom Implementation

### Instructions:

  1. Read in the train_data.
  2. Vectorize train_data using sklearns built in tfidf vectorizer.
  3. Ignore unigrams and make use of both **bigrams & trigrams** and also limit the **max features** to **4000** and **minimum document frequency** to **10**.
  4. After the tfidf vectors are generated as mentioned above, next task is to generate random hyperplanes.
  5. Generate **5 random hyperplanes**. And generate the hyperplanes using a random normal distribution with **mean zero and variance 1**. 
  6. We have set the **numpy random seed to zero**, please do not change it. And then you can make use of **np.random.normal** to generate the vectors for hyperplanes.
  7. As mentioned in the course videos, compute the hash function and also the corresponding hash table for it.
  8. Once the hash table is generated now take in each of the query points from the test data.
  9. Vectorize those query points using the same tfidf vectorizer as mentioned above.
  10. Now use the hash function on this query point and fetch all the similar data points from the hashtable.
  11. Use cosine similarity to compute **11-Nearest Neighbours** from the list of data points obtained in the above step.
  12. Take a majority vote among the 11-Nearest Neighbours and predict the class label for the query point in the test data.
  13. **In case of a tie** in the obtained labels from nearest neighbours, you can pick a label after sorting all the labels **alphabetically**(A-Z), i.e. for example labels starting with A would get more preference than labels starting with Z.
  14. Repeat steps 9 to 13 for all the points in the test data and then finally return a list with all the predicted labels.
  15. Note that there are a total of 10 data points in the test data so the final list you return should be of length 10.
  16. Also note that the cosine similarity function should be written from scratch, you should not directly make use of existing libraries.
  17. Please use the formula of cosine similarity as explained in the course videos, you can make use of numpy or scipy to calculate dot or norm or transpose.

In [139]:
from tqdm import tqdm
from collections import Counter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [141]:
'''Hash Function for Train Data'''
def HashfunctionCall(tfidf_train_data,w):
  hashtable = dict()
  for index,data in tqdm(enumerate(tfidf_train_data)):
    key = list()
    for weight in w:
      value = np.dot(data.toarray(),np.transpose(weight))
      if value > 0:
        key.append('+1')
      else:
        key.append('-1')
    if hashtable.get(tuple(key)) == None:
      hashtable[tuple(key)] = [(index,data.toarray())]
    else:
      hashvalue = hashtable[tuple(key)]
      hashvalue.append((index,data.toarray()))
      hashtable[tuple(key)] = hashvalue
  return hashtable

In [142]:
'''Hash Function for Test Data'''
def HashFunctionCall_Test(tfidf_test_data,w,hashtable):
  Neighbours_list = list()
  for data in tqdm(tfidf_test_data):
    key = list()
    for weight in w:
      value = np.dot(data.toarray(),np.transpose(weight))
      if value > 0:
        key.append('+1')
      else:
        key.append('-1')
    Neighbours_list.append(hashtable[tuple(key)])
  return Neighbours_list

In [143]:
'''Cosine Similarity'''
def cosine_sim(tfidf_test_data,Neighbours_list,no_of_neighbours):
  neighbour_list = list()
  for index in range(tfidf_test_data.shape[0]):
    neighbour_dict = dict()
    neighbours = list()
    for neighbour in Neighbours_list[index]:
      numr = np.dot(tfidf_test_data[index].toarray(),np.transpose(neighbour[1]))
      denom = np.linalg.norm(tfidf_test_data[index].toarray())*np.linalg.norm(neighbour[1])
      cos_sim = numr.ravel()[0]/denom
      neighbour_dict[cos_sim] = neighbour[0]
    for key in sorted(neighbour_dict.keys(),reverse=True)[:no_of_neighbours]:
      neighbours.append(neighbour_dict[key])
    neighbour_list.append(neighbours)
  return neighbour_list

In [144]:
'''Majority Vote'''
def Majority_vote(neigh,train_data):
  Pred_Target_list = list()
  for points in neigh:
    label_list = list()
    Majority_list = list()
    for i in points:
      label = train_data.iloc[i]
      label_list.append(label)
    label_dict = Counter(label_list)
    Major = sorted(label_dict.values(),reverse=True)
    highestval = Major[0]
    Majority_list.append(highestval)
    for index in range(1,len(Major)):
      if Major[index] == highestval:
        Majority_list.append(Major[index])
      else:
        break
    Majority_value = Majority_list[0]
    label_keys = label_dict.keys()
    Majority_key = list()
    for i in range(len(Majority_list)):
      for key in label_keys:
        if label_dict[key] == Majority_value and key not in Majority_key:
          Majority_key.append(key)
    Pred_Target_list.append(sorted(Majority_key)[0])
  return Pred_Target_list

In [145]:
'''Custom LSH Implementation'''

def predictLabels (test_data):
  np.random.seed(0)
  Vectorizer = TfidfVectorizer(ngram_range=(2,3),min_df=10,max_features=4000)
  x_train_tfidf_transform_data = Vectorizer.fit_transform(x_train['text'].values)
  Weight_vector_list= [np.random.normal(loc= 0,scale=1,size = 4000) for i in range(5)]
  hashtable = HashfunctionCall(x_train_tfidf_transform_data,Weight_vector_list)
  x_test_tfidf_transform_data = Vectorizer.transform(test_data['text'].values)
  Neighbours_list = HashFunctionCall_Test(x_test_tfidf_transform_data,Weight_vector_list,hashtable)
  neighbourhood_list = cosine_sim(x_test_tfidf_transform_data,Neighbours_list,11)
  predictLabels_list = Majority_vote(neighbourhood_list,y_train)
  return predictLabels_list


## Grader Cell

Please execute the following Grader cell to verify the correctness of your above implementation. This cell will print "Success" if your implmentation of the predictLabels() is correct, else, it will print "Failed". Make sure you get a "Success" before you submit the code in the classroom.

In [146]:
###########################################
## GRADER CELL: Do NOT Change this.
# This cell will print "Success" if your implmentation of the predictLabels() is correct and the accuracy obtained is above 80%.
# Else, it will print "Failed"
###########################################
import numpy as np

# Predict the labels using the predictLabels() function
Y_custom = np.array(predictLabels(test_data))

# Reference grader array - DO NOT MODIFY IT
Y_grader = np.array(['tech', 'entertainment', 'tech', 'sport', 'business', 'business', 'politics', 'entertainment', 'politics', 'sport'])

# Calculating accuracy by comparing Y_grader and Y_custom
accuracy = np.sum(Y_grader==Y_custom) * 10

if accuracy >= 80:
  print("******** Success ********","Accuracy Achieved = ", accuracy,'%')
else:
  print("####### Failed #######","Accuracy Achieved = ", accuracy,'%')
  print("\nY_grader = \n\n", Y_grader)
  print("\n","*"*50)
  print("\nY_custom = \n\n", Y_custom)


2215it [00:00, 5850.30it/s]
10it [00:00, 2021.06it/s]


******** Success ******** Accuracy Achieved =  90 %
